In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import (recall_score, roc_curve, confusion_matrix, accuracy_score,
                             classification_report, precision_score, f1_score)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [23]:
df = pd.read_parquet('data/compras.parquet', engine='pyarrow')
df.head()

,A3D,BAR-GOURMET,COMUNICACIONES,CORPORATIVA,CUIDADO_PERSONAL,ELECTRO,ESCRITURA Y ACCESORIOS,EVENTOS,GIFTCARD,HOGAR,IMAGEN,LIBRERIA,LINEA_BLANCA,MUNDO APPLE,MUNDO_BEBE,MUNDO_VIAJES,PERFUMES,PRODUCTOS DESCONTINUADOS,Pyme,RELOJES_ JOYAS_REGALOS,SOHO,SONIDO,TIEMPO_LIBRE,VENTA_EMPLEADOS,VIDEO_Y_CONSOLAS,SUCURSAL,FECHA,ANO,MES,DIA,HORA,CANTIDAD_TOTAL,REGION_CLIENTE,REGION_DESPACHO,CANTIDADCANJES,$_RECAUDACIONDESCUENTO,RECAUDACIONDP,$_RECAUDACIONDP,$_RECAUDACIONTC,$_RECAUDACIONDEBITO,NRO_CUOTAS,DOLARES_PREMIO,$_TC,$_DP,$_BRUTO,$_PRECIO_REF,PESO_KG_TOTAL,$_FLETE_TOTAL,SALDO_DP,NUMERO_CUOTAS,TIPO_DTE,TIPO_CLIENTE,TIPO_CAMBIO,TIENDA,FreqTransaccionAnualxVenta,FrecuenciaTransaccionesHistorico,NUEVA_COMPRA_EM_4_MESES,%DP_USADO,%NETO_VS_COSTO,DIF_NETO_COSTO,%DP_USADO_VS_SALDODP,%SALDODP_PESOS_VS_BRUTO,%DP_VS_COSTO
59020,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VENTA WEB,2015-01-01 01:00:00,2015,1,1,1,1,Región Metropolitana,Región Metropolitana,0,0,443.33,269988,3950,0,0,443.33,2,269988,269990,0,12.0,3950,1011.64,0,Boleta,Clasico,609.0,DF,3,8,0,1.042465,1.278907,49479,0.438229,2.281895,1.437429
338784,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,VENTA WEB,2015-01-01 03:00:00,2015,1,1,3,1,Región Metropolitana,Región Metropolitana,1,0,49.10,29902,2900,0,0,49.10,0,29902,29900,0,2.0,2900,202.50,0,Boleta,Clasico,609.0,DF,2,8,0,0.314791,0.303887,-57556,0.242469,4.124498,1.495474
361047,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VENTA WEB,2015-01-01 04:00:00,2015,1,1,4,2,VIII Región,VIII Región,1,0,59.66,36333,15148,0,0,59.66,9647,36333,22990,0,2.0,5500,139.50,0,Boleta,Clasico,609.0,DF,9,22,0,0.660720,0.514172,-18254,0.427670,3.695324,3.177906
157061,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,VENTA WEB,2015-01-01 07:00:00,2015,1,1,7,2,Región Metropolitana,Región Metropolitana,2,0,26.34,16041,20091,0,0,26.34,13541,16041,29580,0,20.0,6550,76.35,0,Boleta,Clasico,609.0,DF,2,16,0,0.180236,0.474777,-27497,0.344990,1.571912,0.954651
450013,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,VENTA WEB,2015-01-01 09:00:00,2015,1,1,9,1,Región Metropolitana,Región Metropolitana,1,0,623.84,379919,30671,0,3,623.84,20071,379919,399990,0,70.0,10600,623.85,3,Boleta,Clasico,609.0,DF,4,17,0,3.799190,4.309308,258126,0.999984,0.949835,1.329750


In [24]:
df.drop(columns=['SUCURSAL','REGION_CLIENTE', 'TIPO_DTE', 'TIPO_CLIENTE'], inplace=True)

In [25]:
categorical = list(df.select_dtypes(include='object').columns)
categorical

['REGION_DESPACHO', 'TIENDA']

In [26]:
y = df['NUEVA_COMPRA_EM_4_MESES']
df.drop(columns=['NUEVA_COMPRA_EM_4_MESES'], inplace=True)

In [27]:
enc = OneHotEncoder(handle_unknown='ignore')
data = enc.fit_transform(df[categorical]).toarray()

In [28]:
data.shape

(543090, 19)

In [29]:
df_features = pd.DataFrame(data)

In [30]:
df.drop(columns=categorical, inplace=True)
df.drop(columns=['FECHA'], inplace=True)

In [31]:
df_merged = pd.concat([df, df_features], axis=1)
df_merged.dropna(inplace=True)
df_merged.reset_index(drop=True, inplace=True)
df_merged.head()

,A3D,BAR-GOURMET,COMUNICACIONES,CORPORATIVA,CUIDADO_PERSONAL,ELECTRO,ESCRITURA Y ACCESORIOS,EVENTOS,GIFTCARD,HOGAR,IMAGEN,LIBRERIA,LINEA_BLANCA,MUNDO APPLE,MUNDO_BEBE,MUNDO_VIAJES,PERFUMES,PRODUCTOS DESCONTINUADOS,Pyme,RELOJES_ JOYAS_REGALOS,SOHO,SONIDO,TIEMPO_LIBRE,VENTA_EMPLEADOS,VIDEO_Y_CONSOLAS,ANO,MES,DIA,HORA,CANTIDAD_TOTAL,CANTIDADCANJES,$_RECAUDACIONDESCUENTO,RECAUDACIONDP,$_RECAUDACIONDP,$_RECAUDACIONTC,$_RECAUDACIONDEBITO,NRO_CUOTAS,DOLARES_PREMIO,$_TC,$_DP,$_BRUTO,$_PRECIO_REF,PESO_KG_TOTAL,$_FLETE_TOTAL,SALDO_DP,NUMERO_CUOTAS,TIPO_CAMBIO,FreqTransaccionAnualxVenta,FrecuenciaTransaccionesHistorico,%DP_USADO,%NETO_VS_COSTO,DIF_NETO_COSTO,%DP_USADO_VS_SALDODP,%SALDODP_PESOS_VS_BRUTO,%DP_VS_COSTO,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2015,4,30,13,2,2,0,68.92,42593,102588,0,0,68.92,99388,42593,141980,0,3.9,3200,181.00,0,618.0,2,12,0.157758,0.635207,-68518,0.380773,0.787843,0.445692,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,10,27,20,1,1,0,31.25,21594,18346,0,0,31.25,14396,21594,35990,0,10.0,3950,143.58,0,691.0,2,12,0.722207,1.512528,10248,0.217649,2.756704,1.070972,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,2016,1,12,13,4,4,0,60.97,43959,13601,0,0,60.97,11001,43959,54960,0,0.8,2600,112.33,0,721.0,3,12,1.912092,4.039447,34750,0.542776,1.473616,1.359697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,7,18,18,1,0,0,0.00,0,193940,0,0,0.00,189990,0,189990,0,10.0,3950,98.36,0,650.0,3,12,0.000000,9.501577,142852,0.000000,0.336512,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2016,11,15,7,4,3,0,98.35,64518,66163,0,0,98.35,62263,64517,96880,34900,7.9,3900,98.36,0,656.0,3,12,0.161297,0.284946,-204296,0.999898,0.666021,1.027292,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [32]:
df_merged.shape

(543090, 74)

In [33]:
x = df_merged
sc = MinMaxScaler()
x = sc.fit_transform(x)

In [34]:
tscv = TimeSeriesSplit(n_splits=5)

Accuracy, Recall, Precission, Fbeta

In [35]:
def trainning(model, model_name):
    results = []
    iteration = 1

    # Cross-validation para TimeSeries
    for train_index, test_index in tscv.split(x):
        print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = model.fit(x_train, y_train)
        model_name = model_name
        
        # Generar tabla umbral
        for umbral in range(1, 10):
            umbral = umbral / 10
            y_pred = (model.predict_proba(x_test)[:,1] >= umbral).astype(int)
            accuracy = accuracy_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            f1_score = 2 * precision * recall / (precision + recall)
            cm = confusion_matrix(y_test, y_pred)
            (tn, fp, fn, tp) = cm.flatten()
            results.append([model_name, umbral, accuracy, recall, precision, f1_score, tn, tp, fn, fp])
        iteration += 1

    return pd.DataFrame(results, columns=["model_name", "umbral", "accuracy", "recall", "precision", "f1_score", "tn", "tp", "fn", "fp"])

In [36]:
models = []
models.append(("LogisticRegression", LogisticRegression(max_iter=1000, solver='liblinear')))
models.append(("GaussianNB", GaussianNB()))
models.append(("RandomForestClassifier", RandomForestClassifier()))
models.append(("DecisionTreeClassifier", DecisionTreeClassifier()))
models.append(("xgb.XGBClassifier", xgb.XGBClassifier(eval_metric='mlogloss')))

In [37]:
result = pd.DataFrame()
for i in range(0,len(models)):
    result = result.append(trainning(models[i][1], models[i][0]))

TRAIN: [    0     1     2 ... 90512 90513 90514] TEST: [ 90515  90516  90517 ... 181027 181028 181029]
TRAIN: [     0      1      2 ... 181027 181028 181029] TEST: [181030 181031 181032 ... 271542 271543 271544]
TRAIN: [     0      1      2 ... 271542 271543 271544] TEST: [271545 271546 271547 ... 362057 362058 362059]
TRAIN: [     0      1      2 ... 362057 362058 362059] TEST: [362060 362061 362062 ... 452572 452573 452574]
TRAIN: [     0      1      2 ... 452572 452573 452574] TEST: [452575 452576 452577 ... 543087 543088 543089]
TRAIN: [    0     1     2 ... 90512 90513 90514] TEST: [ 90515  90516  90517 ... 181027 181028 181029]
TRAIN: [     0      1      2 ... 181027 181028 181029] TEST: [181030 181031 181032 ... 271542 271543 271544]
TRAIN: [     0      1      2 ... 271542 271543 271544] TEST: [271545 271546 271547 ... 362057 362058 362059]
TRAIN: [     0      1      2 ... 362057 362058 362059] TEST: [362060 362061 362062 ... 452572 452573 452574]
TRAIN: [     0      1      2 ..

In [38]:
result_df = result.groupby(["model_name","umbral"]).mean()
result_df.sort_values(by=['f1_score'], ascending=False)

,,accuracy,recall,precision,f1_score,tn,tp,fn,fp
model_name,umbral,,,,,,,,
xgb.XGBClassifier,0.4,0.765347,0.846557,0.682081,0.755456,36114.2,33161.2,5907.0,15332.6
RandomForestClassifier,0.4,0.755919,0.854933,0.667779,0.749848,34922.8,33499.2,5569.0,16524.0
xgb.XGBClassifier,0.3,0.730416,0.932952,0.623895,0.747579,29627.2,36486.4,2581.8,21819.6
LogisticRegression,0.3,0.742021,0.866768,0.648841,0.742033,33153.4,34010.6,5057.6,18293.4
RandomForestClassifier,0.3,0.719596,0.941307,0.612277,0.741817,28312.6,36821.6,2246.6,23134.2
xgb.XGBClassifier,0.5,0.774181,0.745956,0.731949,0.738761,40805.4,29269.6,9798.6,10641.4
RandomForestClassifier,0.5,0.769335,0.747775,0.722749,0.734852,40269.6,29366.8,9701.4,11177.2
xgb.XGBClassifier,0.2,0.689866,0.986130,0.581510,0.731216,23907.8,38535.4,532.8,27539.0
LogisticRegression,0.4,0.763100,0.736325,0.717316,0.726227,40107.6,28964.4,10103.8,11339.2
